In [182]:
import numpy as np

class BasicNNModule:
    def __init__(self):
        self.lr = 0.001

class SingleLinear(BasicNNModule):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weights = np.ones((out_features, in_features)) * 0.5
        self.bias = np.zeros(out_features)

    def forward(self, x):
        return np.matmul(x, self.weights.T) + self.bias
    
    def backward(self, delta_next_layer, w_next_layer, b_next_layer, a_cur_layer, y_last_layer):
        # print(d2y.shape, y2i.shape, y_last_layer.shape)
        # delta = d2y * y2i
        delta = (np.matmul(delta_next_layer.T, w_next_layer) * a_cur_layer).T
        # print(delta)
        # L2 regularization
        self.weights += self.lr * (np.matmul(delta, y_last_layer) - 0.01 * self.weights)
        # self.weights += self.lr * np.matmul(delta, y_last_layer)
        self.bias += self.lr * delta.T[0]
        return delta, self.weights, self.bias
    
    def softmax_backward(self, y, label, y_last_layer):
        delta = (y - label).T
        # print('before update:', self.weights, self.bias)
        # print(delta.shape, y_last_layer.shape, self.bias.shape)

        # L2 regularization
        self.weights += self.lr * (np.matmul(delta, y_last_layer) - 0.01 * self.weights)
        # self.weights += self.lr * np.matmul(delta, y_last_layer)

        self.bias += self.lr * delta.T[0]
        # print('after update:', self.weights, self.bias)
        return delta, self.weights, self.bias

class relu:
    def forward(self, x):
        return np.maximum(x, 0)
    
    def backward(self, x):
        return np.where(x > 0, 1, 0)
    
class Softmax:
    def forward(self, x):
        # 減去最大值以防止 overflow
        x = x - np.max(x, axis=1, keepdims=True)
        exp_x = np.exp(x)
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)
    


In [ ]:
import numpy as np

with open('Iris.csv') as f:
    data = f.readlines()[1:]

data = [line.strip().split(',')[1:] for line in data]
data = np.array(data)
labels = []

classes = np.unique(data[:, 4])

for d in data:
    for cls in classes:
        if d[4] == cls:
            one_hot = np.zeros(len(classes))
            one_hot[classes.tolist().index(cls)] = 1
            labels.append(one_hot)

inputs = data[:, :4].astype(np.float32)

# inputs = (inputs - np.mean(inputs, axis=0)) / np.std(inputs, axis=0)

labels = np.array(labels)

# 打亂數據
idx = np.random.permutation(len(inputs))
inputs = inputs[idx]
labels = labels[idx]


linear_1 = SingleLinear(4, 5)
relu_1 = relu()
linear_2 = SingleLinear(5, 5)
relu_2 = relu()
linear_3 = SingleLinear(5, 3)
softmax_3 = Softmax()

for _ in range(5):

    for input, label in zip(inputs, labels):
        input = input.reshape(1, -1)
        # print(input, label)
        i_1 = linear_1.forward(input)
        y_1 = relu_1.forward(i_1)
        i_2 = linear_2.forward(y_1)
        y_2 = relu_2.forward(i_2)
        i_3 = linear_3.forward(y_2)
        print("i_3", i_3)
        y_3 = softmax_3.forward(i_3)
        print("y_3", y_3)
        print("label", label)
        delta_3, w_3, b_3 = linear_3.softmax_backward(y_3, label, y_2)
        delta_2, w_2, b_2 = linear_2.backward(delta_3, w_3, b_3, relu_2.backward(i_2), y_1)
        delta_1, w_1, b_1 = linear_1.backward(delta_2, w_2, b_2, relu_1.backward(i_1), input)
        # print('delta_3:\n', delta_3)
        # print('w_3:\n', w_3)
        # print('b_3:\n', b_3)
        
        # print('delta_2:\n', delta_2)
        # print('w_2:\n', w_2)
        # print('b_2:\n', b_2)
        # break
        # delta_1 = linear_1.backward(- ())
    

In [135]:
np.matmul(delta_3.T, w_3)

array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]])

In [171]:
class Softmax:
    def forward(self, x):
        # 減去最大值以防止 overflow
        x = x - np.max(x, axis=1, keepdims=True)
        exp_x = np.exp(x)
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)
Softmax().forward(np.array([[1, 5, 2]]))

array([[0.01714783, 0.93623955, 0.04661262]])

In [236]:
a = np.random.randn(3, 4)
b = np.random.randn(4)
print(a)
print(b)
print(a.dot(b))

[[-0.95799297  0.50648367 -1.29696496  0.99630143]
 [-0.48203369  0.28668749  2.12959588  0.65556684]
 [-0.08816886  0.089359   -0.41808245  0.38364675]]
[ 0.09246073  0.36246263  0.09009849 -0.19246959]
[-0.21360764  0.12504099 -0.08727179]


In [264]:
import numpy as np

with open('Iris.csv') as f:
    data = f.readlines()[1:]

data = [line.strip().split(',')[1:] for line in data]
data = np.array(data)
labels = []

classes = np.unique(data[:, 4])

for d in data:
    for cls in classes:
        if d[4] == cls:
            one_hot = np.zeros(len(classes))
            one_hot[classes.tolist().index(cls)] = 1
            labels.append(one_hot)

inputs = data[:, :4].astype(np.float32)

# inputs = (inputs - np.mean(inputs, axis=0)) / np.std(inputs, axis=0)

labels = np.array(labels)

# 打亂數據
idx = np.random.permutation(len(inputs))
inputs = inputs[idx].T
labels = labels[idx].T


inputs.shape, labels.shape

((4, 150), (3, 150))

In [5]:
m, n = inputs.T.shape

print('m:', m, 'n:', n)

def init_params():
    W1 = np.random.rand(10, 4) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(3, 10) - 0.5
    b2 = np.random.rand(3, 1) - 0.5
    return W1, b1, W2, b2

def relu(Z):
    return np.maximum(Z, 0)

def relu_deriv(Z):
    return Z > 0

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    # print('X shape:', X.shape, 'W1 shape:', W1.shape)
    Z1 = np.matmul(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.matmul(W2, A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2



def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, A2, W2, X, Y):
    one_hot_Y = one_hot(Y)
    print('A2 shape:', A2.shape, 'Y shape:', Y.shape, 'one_hot_Y shape:', one_hot_Y.shape)
    dZ2 = A2 - one_hot_Y
    # dW2 = 1 / m * dZ2.dot(A1.T)
    # print('dZ2 shape:', dZ2.shape, 'A1 shape:', A1.shape)
    dW2 = (1 / m) * np.matmul(dZ2, A1.T)
    # print(dW2.shape)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * relu_deriv(Z1)
    # dW1 = 1 / m * dZ1.dot(X.T)
    dW1 = (1 / m) * np.matmul(dZ1, X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    # print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        # for X, Y in zip(X_all.T, Y_all.T):
        #     X = X.reshape(-1, 1)
        #     Y = Y.reshape(-1, 1)
            # print('Y shape:', Y.shape, 'Y_all shape:', Y_all.shape)
            # print('X_all shape:', X_all.shape)
            # print('X shape:', X.shape)
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)

            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
        break
    return W1, b1, W2, b2

m: 4 n: 150


In [6]:
X_train = inputs
Y_train = np.argmax(labels, axis=0)
# Y_train = labels
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 150 is different from 4)

In [473]:
m, n = inputs.T.shape

print('m:', m, 'n:', n)

def init_params():
    W1 = np.random.rand(10, 4) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(3, 10) - 0.5
    b2 = np.random.rand(3, 1) - 0.5
    return W1, b1, W2, b2

def relu(Z):
    return np.maximum(Z, 0)

def relu_deriv(Z):
    return Z > 0

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    # print('X shape:', X.shape, 'W1 shape:', W1.shape)
    Z1 = np.matmul(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.matmul(W2, A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2



def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, A2, W2, X, Y):
    # one_hot_Y = Y.reshape(-1, 1)
    one_hot_Y = one_hot(Y)
    # print('A2 shape:', A2.shape, 'Y shape:', Y.shape, 'one_hot_Y shape:', one_hot_Y.shape)
    dZ2 = A2 - one_hot_Y
    # dW2 = 1 / m * dZ2.dot(A1.T)
    # print('dZ2 shape:', dZ2.shape, 'A1 shape:', A1.shape)
    dW2 = (1 / m) * np.matmul(dZ2, A1.T)
    # print(dW2.shape)
    db2 = (1 / m) * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * relu_deriv(Z1)
    # dW1 = 1 / m * dZ1.dot(X.T)
    dW1 = (1 / m) * np.matmul(dZ1, X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    # print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    bs = 10
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        # for X, Y_s in zip(X_all.T, Y_all.T):

        #     X = X.reshape(-1, 1)
        #     Y = Y.reshape(-1, 1)
        # print('Y shape:', Y.shape, 'Y_all shape:', Y_all.shape)
        # print('X_all shape:', X_all.shape)
        # print('X shape:', X.shape)
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 50 == 0:
            print("Iteration: ", i)

            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
        # break
    return W1, b1, W2, b2

m: 150 n: 4


In [476]:
X_train = inputs
Y_train = np.argmax(labels, axis=0)
# Y_train = labels
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.05, 101)

Iteration:  0
0.3333333333333333
Iteration:  50
0.9133333333333333
Iteration:  100
0.9666666666666667


# =======================================

In [ ]:
def init_params():
    W1 = np.ones((4, 10))
    b1 = np.zeros((1, 10))
    W2 = np.ones((10, 3))
    b2 = np.zeros((1, 3))
    return W1, b1, W2, b2

def relu(x):
    return np.maximum(x, 0)

def d_relu(x):
    return x > 0

def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def forward_prop(W1, b1, W2, b2, X):

    # X shape: (bs, n_features)
    I1 = np.matmul(X, W1) + b1  
    Y1 = relu(I1)

    I2 = np.matmul(Y1, W2) + b2
    Y2 = softmax(I2)
    return I1, Y1, I2, Y2

def backward_prop(I1, Y1, Y2, W2, X, Y):

    # 設定標準化因子 (batch size的倒數) 將梯度標準化，避免 batch size 過大造成梯度累積過大
    m = X.shape[0]
    norm_factor = 1 / m
    
    delta_2 = Y2 - Y  # (bs, 3)

    # Y1.T shape -> (n_layer1, bs)
    # delta_2 shape -> (bs, n_layer2)

    dW2 = norm_factor * np.matmul(Y1.T, delta_2)  
    db2 = norm_factor * np.sum(delta_2, axis=0, keepdims=True)

    # dW2 shape -> (n_layer1, n_layer2)
    # db2 shape -> (1, n_layer2)

    # ========================================

    # delta_2 shape -> (bs, n_layer2)
    # W2.T shape -> (n_layer2, n_layer1)
    # d_relu(I1) shape -> (bs, n_layer1)

    delta_1 = np.matmul(delta_2, W2.T) * d_relu(I1)  
 
    # X.T shape -> (n_features, bs)
    # delta_1 shape -> (bs, n_layer1)
    
    dW1 = norm_factor * np.matmul(X.T, delta_1)  
    db1 = norm_factor * np.sum(delta_1, axis=0, keepdims=True)

    # dW1 shape -> (n_features, n_layer1)
    # db1 shape -> (1, n_layer1)
    
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, lr):
    W1 = W1 - lr * dW1
    b1 = b1 - lr * db1
    W2 = W2 - lr * dW2
    b2 = b2 - lr * db2
    return W1, b1, W2, b2

def get_predictions(A2):
    return np.argmax(A2, axis=1)

def get_accuracy(predictions, Y):
    Y = np.argmax(Y, axis=1)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X_all, Y_all, X_val, Y_val, alpha, iterations, bs):
    # X_all -> (n_samples, n_features)
    # Y_all -> (n_samples, n_classes) one-hot 
    
    W1, b1, W2, b2 = init_params()
    
    # 將全部的資料打包成 batch，每個 batch 的大小為 bs
    X_batch = X_all.reshape(-1, bs, X_all.shape[1])
    Y_batch = Y_all.reshape(-1, bs, Y_all.shape[1])
    
    for i in range(iterations):
        loss = 0
        for X, Y in zip(X_batch, Y_batch):
          
            # print('Y shape:', Y.shape, 'Y_all shape:', Y_all.shape)
            I1, Y1, I2, Y2 = forward_prop(W1, b1, W2, b2, X)
            dW1, db1, dW2, db2 = backward_prop(I1, Y1, Y2, W2, X, Y)
            W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
            loss += -np.sum(Y * np.log(Y2))

        if i % 1 == 0:
            print("Iteration: ", i)
            print("Loss: ", loss)
            _, _, _, pred = forward_prop(W1, b1, W2, b2, X_val)
            predictions = get_predictions(pred)
            print('Acc:', get_accuracy(predictions, Y_val))
            # print(predictions, Y)
        break

    return W1, b1, W2, b2

In [36]:
import numpy as np

# 固定隨機種子，確保同參數下每次結果都相同
np.random.seed(1)

with open('Iris.csv') as f:
    data = f.readlines()[1:]

data = [line.strip().split(',')[1:] for line in data]
data = np.array(data)
labels = []

classes = np.unique(data[:, 4])

for d in data:
    for cls in classes:
        if d[4] == cls:
            one_hot = np.zeros(len(classes))
            one_hot[classes.tolist().index(cls)] = 1
            labels.append(one_hot)

inputs = data[:, :4].astype(np.float32)

# inputs = (inputs - np.mean(inputs, axis=0)) / np.std(inputs, axis=0)

labels = np.array(labels)

# 打亂數據
idx = np.random.permutation(len(inputs))

X_data = inputs[idx]
# Y_train = np.argmax(labels, axis=0)

Y_data = labels[idx]

train_size = int(len(X_data) * 0.8)
val_size = len(X_data) - train_size

X_train, Y_train = X_data[:train_size], Y_data[:train_size]
X_val, Y_val = X_data[val_size:], Y_data[val_size:]

print(X_train.shape)

batch_size = 120
lr = 0.05

W1, b1, W2, b2 = gradient_descent(X_train, Y_train, X_val, Y_val, lr, 30, batch_size)
# W1, b1, W2, b2

(120, 4)
d_relu(I1): (120, 10)
Iteration:  0
Loss:  131.83347464017316
Acc: 0.36666666666666664


In [24]:
x = np.random.randn(4, 3)
y = np.random.randn(3, 4)

print(np.matmul(y.T, x.T))
print(np.matmul(x, y).T)


[[-2.42146685 -1.49794256 -0.89849127 -0.66365005]
 [-0.51160228  0.93826069 -0.27637929 -0.5548736 ]
 [-0.35771471 -1.06233953 -0.41493013 -0.36709497]
 [ 3.28614577  4.57316731  1.2664861   0.41866526]]
[[-2.42146685 -1.49794256 -0.89849127 -0.66365005]
 [-0.51160228  0.93826069 -0.27637929 -0.5548736 ]
 [-0.35771471 -1.06233953 -0.41493013 -0.36709497]
 [ 3.28614577  4.57316731  1.2664861   0.41866526]]


## =================== 改良 =====================


In [46]:
def init_params():
    params = {'w': [], 'b': [], 'act_func': []}
    W1 = np.random.randn(4, 10) * np.sqrt(2/4)
    b1 = np.zeros((1, 10))
    
    W2 = np.random.randn(10, 5) * np.sqrt(2/10)
    b2 = np.zeros((1, 5))
    
    W3 = np.random.randn(5, 3) * np.sqrt(2/5)
    b3 = np.zeros((1, 3))
    # W3 = np.ones((5, 3))
    # b3 = np.zeros((1, 3))

    params['w'].append(W1)
    params['b'].append(b1)
    params['act_func'].append(relu)
    params['w'].append(W2)
    params['b'].append(b2)
    params['act_func'].append(relu)
    params['w'].append(W3)
    params['b'].append(b3)
    params['act_func'].append(softmax)
    return params

def relu(x):
    return np.maximum(x, 0)

def d_relu(x):
    return x > 0

def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def forward_prop(params, X):
    forward_val = {'I': [], 'Y': []}
    # X shape: (bs, n_features)

    for idx, (w, b, act_func) in enumerate(zip(params['w'], params['b'], params['act_func'])):
        # print('forward idx:', idx, 'X shape:', X.shape, 'w', w[0])
        if idx == 0:
            I = np.matmul(X, w) + b
        else:
            I = np.matmul(Y, w) + b
        Y = act_func.f(I)
        # print('idx:', idx, 'I shape:', I.shape, 'Y shape:', Y.shape)
        forward_val['I'].append(I)
        forward_val['Y'].append(Y)

    # I1 = np.matmul(X, W1) + b1  
    # Y1 = relu(I1)

    # I2 = np.matmul(Y1, W2) + b2
    # Y2 = softmax(I2)
    return forward_val

def backward_prop(params: dict, forward_val: dict, input_feat, label) -> dict:

    params_delta = {'W': [], 'b': []}

    # 設定標準化因子 (batch size的倒數) 將梯度標準化，避免 batch size 過大造成梯度累積過大
    m = input_feat.shape[0]
    norm_factor = 1 / m
    n_layer = len(params['w'])

    for idx, (Y, I, W, act_func) in enumerate(zip(forward_val['Y'][::-1], forward_val['I'][::-1], params['w'][::-1], params['act_func'][::-1])):
        # 由 output layer 開始計算 delta params

        # 當目前為 output layer 時
        # delta 為 dL/dY * dY/dI -> Cross Entropy Loss 與 Softmax 的導函數相乘
        # 可化簡為 Y - label
        
        if idx == 0:
            delta = Y - label
        else:
            # print('idx', idx, 'delta shape:', delta.shape, 'W shape:', W.shape)
            delta = np.matmul(delta, next_layer_w.T) * act_func.b(I)
       

        # 當目前為 input layer 時，dW 的左矩陣為 input_feat.T
        if idx == n_layer - 1:
            dW = norm_factor * np.matmul(input_feat.T, delta)
        else:
            dW = norm_factor * np.matmul(forward_val['Y'][::-1][idx+1].T, delta)
            
        db = norm_factor * np.sum(delta, axis=0, keepdims=True)

        
        params_delta['W'].append(dW)
        params_delta['b'].append(db)
        next_layer_w = W
    
    # delta_2 = Y2 - Y  # (bs, 3)

    # # Y1.T shape -> (n_layer1, bs)
    # # delta_2 shape -> (bs, n_layer2)

    # dW2 = norm_factor * np.matmul(Y1.T, delta_2)  
    # db2 = norm_factor * np.sum(delta_2, axis=0, keepdims=True)

    # # dW2 shape -> (n_layer1, n_layer2)
    # # db2 shape -> (1, n_layer2)

    # # ========================================

    # # delta_2 shape -> (bs, n_layer2)
    # # W2.T shape -> (n_layer2, n_layer1)
    # # d_relu(I1) shape -> (bs, n_layer1)

    # delta_1 = np.matmul(delta_2, W2.T) * d_relu(I1)  
 
    # # X.T shape -> (n_features, bs)
    # # delta_1 shape -> (bs, n_layer1)
    
    # dW1 = norm_factor * np.matmul(X.T, delta_1)  
    # db1 = norm_factor * np.sum(delta_1, axis=0, keepdims=True)

    # # dW1 shape -> (n_features, n_layer1)
    # # db1 shape -> (1, n_layer1)
    
    # params_delta['W'].append(dW1)
    # params_delta['b'].append(db1)
    # params_delta['W'].append(dW2)
    # params_delta['b'].append(db2)

    return params_delta

def update_params(params: dict, params_delta: dict, lr) -> dict:
    # W1 = W1 - lr * dW1
    # b1 = b1 - lr * db1
    # W2 = W2 - lr * dW2
    # b2 = b2 - lr * db2
    
    for idx, (W, b, dW, db) in enumerate(zip(params['w'], params['b'], params_delta['W'][::-1], params_delta['b'][::-1])):
        # print('idx:', idx, 'W shape:', W.shape, 'b shape:', b.shape, 'dW shape:', dW.shape, 'db shape:', db.shape)
        W = W - lr * dW
        b = b - lr * db
        params['w'][idx] = W
        params['b'][idx] = b

    return params

def get_predictions(A2):
    return np.argmax(A2, axis=1)

def get_accuracy(predictions, Y):
    Y = np.argmax(Y, axis=1)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(params, X_all, Y_all, X_val, Y_val, alpha, iterations, bs):
    # X_all -> (n_samples, n_features)
    # Y_all -> (n_samples, n_classes) one-hot 
    
    # params = init_params()
    
    # 將全部的資料打包成 batch，每個 batch 的大小為 bs
    X_batch = X_all.reshape(-1, bs, X_all.shape[1])
    Y_batch = Y_all.reshape(-1, bs, Y_all.shape[1])
    
    for i in range(iterations):
        loss = 0
        for X, Y in zip(X_batch, Y_batch):
          
            # print('Y shape:', Y.shape, 'Y_all shape:', Y_all.shape)
            forward_val = forward_prop(params, X)
            params_delta = backward_prop(params, forward_val, X, Y)
            params = update_params(params, params_delta, alpha)
            loss += -np.sum(Y * np.log(forward_val['Y'][-1]))

        if i % 1 == 0:
            print("Iteration: ", i)
            print('Loss:', round(loss, 2))
            pred = forward_prop(params, X_val)
            predictions = get_predictions(pred['Y'][-1])
            print('Val Acc:', round(get_accuracy(predictions, Y_val), 2))
            # print(predictions, Y)
        # break

    return params


class ReLU:
    def f(x):
        return np.maximum(x, 0)
    def b(x):
        return x > 0
    
class Softmax:
    def f(x):
        exp_x = np.exp(x)
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)



In [47]:
import numpy as np

# 固定隨機種子，確保同參數下每次結果都相同
np.random.seed(1)

with open('Iris.csv') as f:
    data = f.readlines()[1:]

data = [line.strip().split(',')[1:] for line in data]
data = np.array(data)
labels = []

classes = np.unique(data[:, 4])

for d in data:
    for cls in classes:
        if d[4] == cls:
            one_hot = np.zeros(len(classes))
            one_hot[classes.tolist().index(cls)] = 1
            labels.append(one_hot)

inputs = data[:, :4].astype(np.float32)

# inputs = (inputs - np.mean(inputs, axis=0)) / np.std(inputs, axis=0)

labels = np.array(labels)

# 打亂數據
idx = np.random.permutation(len(inputs))

X_data = inputs[idx]
# Y_train = np.argmax(labels, axis=0)

Y_data = labels[idx]

train_size = int(len(X_data) * 0.8)
val_size = len(X_data) - train_size

X_train, Y_train = X_data[:train_size], Y_data[:train_size]
X_val, Y_val = X_data[val_size:], Y_data[val_size:]

print(X_train.shape)

batch_size = 3
lr = 0.05


init_params = {
    'w': [
        np.random.randn(4, 5) * np.sqrt(2/4),
        np.random.randn(5, 5) * np.sqrt(2/10),
        np.random.randn(5, 5) * np.sqrt(2/10),
        np.random.randn(5, 3) * np.sqrt(2/5)
        ],
    'b': [
        np.zeros((1, 5)),
        np.zeros((1, 5)),
        np.zeros((1, 5)),
        np.zeros((1, 3))
        ],
    'act_func': [
        ReLU,
        ReLU, 
        ReLU,
        Softmax
                 ]
    }

params = gradient_descent(init_params, X_train, Y_train, X_val, Y_val, lr, 30, batch_size)
# params

(120, 4)
Iteration:  0
Loss: 137.85
Val Acc: 0.69
Iteration:  1
Loss: 84.05
Val Acc: 0.69
Iteration:  2
Loss: 67.35
Val Acc: 0.69
Iteration:  3
Loss: 60.1
Val Acc: 0.69
Iteration:  4
Loss: 55.08
Val Acc: 0.69
Iteration:  5
Loss: 51.87
Val Acc: 0.7
Iteration:  6
Loss: 51.15
Val Acc: 0.72
Iteration:  7
Loss: 50.16
Val Acc: 0.72
Iteration:  8
Loss: 47.68
Val Acc: 0.72
Iteration:  9
Loss: 46.04
Val Acc: 0.72
Iteration:  10
Loss: 44.18
Val Acc: 0.74
Iteration:  11
Loss: 44.2
Val Acc: 0.75
Iteration:  12
Loss: 41.39
Val Acc: 0.84
Iteration:  13
Loss: 40.13
Val Acc: 0.85
Iteration:  14
Loss: 39.35
Val Acc: 0.85
Iteration:  15
Loss: 39.47
Val Acc: 0.84
Iteration:  16
Loss: 39.21
Val Acc: 0.84
Iteration:  17
Loss: 41.41
Val Acc: 0.79
Iteration:  18
Loss: 35.22
Val Acc: 0.75
Iteration:  19
Loss: 37.96
Val Acc: 0.83
Iteration:  20
Loss: 32.62
Val Acc: 0.91
Iteration:  21
Loss: 41.35
Val Acc: 0.74
Iteration:  22
Loss: 37.03
Val Acc: 0.88
Iteration:  23
Loss: 28.11
Val Acc: 0.95
Iteration:  24
Loss

In [21]:
a = {'a': [1, 2], 'b': [3, 4]}

for x, y in reversed(list(zip(a['a'], a['b']))):
    print(x, y)

a = [1, 2, 3]
# inverse a
a[::-1]

2 4
1 3


[3, 2, 1]